In [1]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [2]:
import pandas as pd
df = pd.read_csv('../NIFTY50_all.csv')

In [23]:
from datetime import datetime
from sklearn.impute import SimpleImputer
df1 = df.drop(['Trades', 'Deliverable Volume', 'Series'], axis=1)
df1['Symbol'] = pd.Categorical(df1['Symbol'])
df1['Date'] = [datetime.strptime(x, '%Y-%m-%d') for x in df1['Date']]
df1['Date']=df1['Date'].map(datetime.toordinal)
df2 = pd.get_dummies(data=df1, drop_first=True)

In [29]:
imp = SimpleImputer()
p = imp.fit_transform(df2)
df1['%Deliverble'] = p[:, 10]

array([0.3612, 0.3172, 0.2088, ..., 0.1344, 0.1348, 0.2726])